<a href="https://colab.research.google.com/github/1028Luo/LLM-Domain-Specific-Assistant/blob/main/LLAMA_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [3]:

# install dependencies and import

# llama index
!pip install llama-index-core
!pip install llama-index-llms-openai
!pip install llama-index-llms-replicate
!pip install llama-index-embeddings-huggingface

!pip install "transformers>=4.45.1"
!pip install huggingface_hub
!pip install transformers

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
# load fine-tuned model from hub
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import bitsandbytes
from huggingface_hub import login


In [5]:
import torch
print(torch.cuda.is_available())  # Returns True if a GPU is available


False


In [24]:
!pip install -U bitsandbytes
import bitsandbytes
from IPython import get_ipython
if get_ipython() is not None:
    get_ipython().kernel.do_shutdown(restart=True)

In [4]:


# import any embedding model on HF hub (https://huggingface.co/spaces/mteb/leaderboard)
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
# Settings.embed_model = HuggingFaceEmbedding(model_name="thenlper/gte-large") # alternative model

Settings.llm = None
Settings.chunk_size = 512
Settings.chunk_overlap = 25


model_name = "meta-llama/Llama-3.1-8B-instruct"

# Log in to Hugging Face with your token
from google.colab import userdata
my_hugging_face_token = userdata.get('huggingface_token')
login(token=my_hugging_face_token)


model = AutoModelForCausalLM.from_pretrained(model_name,
                                             load_in_8bit=True,
                                            device_map="cuda",
                                            trust_remote_code=False,
                                            revision="main")


# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# # Manually move model to GPU (optional, ensures GPU-only placement)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# articles available here: {add GitHub repo}
documents = SimpleDirectoryReader("RAG_Data").load_data()

# some ad hoc document refinement
# print(len(documents))
# for doc in documents:
#     if "Member-only story" in doc.text:
#         documents.remove(doc)
#         continue

#     if "The Data Entrepreneurs" in doc.text:
#         documents.remove(doc)

#     if " min read" in doc.text:
#         documents.remove(doc)

print("length of document: ", len(documents))

# store docs into vector DB
index = VectorStoreIndex.from_documents(documents)

# set number of docs to retreive
top_k = 6

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)


def chat():
    while True:
        question = input("You: ")
        if question.lower() == 'exit':
            break
        # query documents

        response = query_engine.query(question)

        # reformat response
        context = "Context:\n"
        for i in range(top_k):
            context = context + response.source_nodes[i].text + "\n\n"

        # # prompt with no context
        # intstructions_string = f"""You are called MyGPT, an expert assistant in Robotics and AI, providing detailed and accurate responses to any query about the knowledge that you have learnt and  in your RAG data base.\
        # MyGPT will tailor the length of its responses to match the query, providing clear explanation on papers, projects and concepts in the field of robotics and AI \
        # and should keep the answer concise.

        # Please respond to the following question.
        # """
        # prompt_template = lambda comment: f'''[INST] {intstructions_string} \n{comment} \n[/INST]'''

        # prompt = prompt_template(question)
        # # print(prompt)

        # model.eval()

        # inputs = tokenizer(prompt, return_tensors="pt")
        # outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

        # print(tokenizer.batch_decode(outputs)[0])


        # ##### with Context #####
        # prompt_template_w_context = lambda context, question: f"""[INST]You are called MyGPT, an expert assistant in Robotics and AI, providing detailed and accurate responses to any query about the knowledge that you have learnt and  in your RAG data base.\
        # MyGPT will tailor the length of its responses to match the query, providing clear explanation on papers, projects and concepts in the field of robotics and AI \
        # and should keep the answer concise.

        # {context}
        # Please respond to the following question. Use the context above if it is helpful.

        # {question}
        # [/INST]
        # """


        prompt = [
            {"role": "system", "content": "You are called MyGPT, an expert assistant in Robotics and AI, providing detailed and accurate responses to any query about the knowledge that you have learnt and  in your RAG data base.\
        MyGPT will tailor the length of its responses to match the query, providing clear explanation on papers, projects and concepts in the field of robotics and AI \
        and should keep the answer concise. You can use the provided context if needed."},
            {"role": "system", "content": context},
            {"role": "user", "content": question},
        ]

        pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
        )

        generation_args = {
            "max_new_tokens": 500,
            "return_full_text": False,
            "temperature": 0.0,
            "do_sample": False,
            "pad_token_id": tokenizer.pad_token_id
        }

        output = pipe(prompt, **generation_args)
        print(output[0]['generated_text'])




chat()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


LLM is explicitly disabled. Using MockLLM.


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend